# 1. Алгоритмы тестов Jarque-Bera и Колмогорова-Смирнова

In [2]:

# Using scipy
import scipy.stats as stats
import numpy as np
sample = np.random.normal(0, 1, 1000)

jb_stat_scipy, jb_p_value_scipy = stats.jarque_bera(sample)
ks_stat_scipy, ks_p_value_scipy = stats.kstest(sample, 'norm')

jb_stat_scipy, jb_p_value_scipy, ks_stat_scipy, ks_p_value_scipy


(0.3603437359564521,
 0.8351266675444148,
 0.02583779141520348,
 0.5083621363689794)

In [3]:

# Using statsmodels for Jarque-Bera
import statsmodels.api as sm

jb_stat_sm, _, jb_p_value_sm, _ = sm.stats.jarque_bera(sample)

jb_stat_sm, jb_p_value_sm


(0.3603437359564521, -0.046456864370736976)

# 2. Проверка остатков модели на нормальность

In [4]:

# Тест Шапиро-Вилька using scipy
shapiro_stat_scipy, shapiro_p_value_scipy = stats.shapiro(sample)

shapiro_stat_scipy, shapiro_p_value_scipy


(0.9987978935241699, 0.7535416483879089)

# 4. Проверка модели на адекватность

In [5]:

# Пример расчета коэффициента детерминации R^2 для линейной регрессии using sklearn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

X = np.random.rand(100, 1) * 10
y = 2.5 * X + np.random.randn(100, 1) * 2

model = LinearRegression().fit(X, y)
y_pred = model.predict(X)

r2_sklearn = r2_score(y, y_pred)

r2_sklearn


0.9365614119098169

# 3. Теоретические условия применения

### Тест Jarque-Bera
Подходит для больших выборок.

### Тест Колмогорова-Смирнова
Не требует, чтобы выборка была большой. Данные должны быть непрерывными.

### Тест Хельвига
Подходит для малых выборок.

### Тест Шапиро-Вилька
Лучше всего использовать для выборок размером менее 50 элементов.

### Тест Хельвига

Тест Хельвига специфичен и не представлен в стандартных библиотеках Python. Однако он важен для малых выборок.

### Тест на автокорреляцию остатков

In [6]:

# Для демонстрации используем statsmodels и функцию acorr_ljungbox
from statsmodels.stats.diagnostic import acorr_ljungbox

# Генерируем остатки (в данном случае просто случайные числа)
residuals = np.random.randn(100)

lbvalue, pvalue = acorr_ljungbox(residuals, lags=[10])
pvalue


'lb_pvalue'

### Тест на гетероскедастичность остатков

In [7]:

# Для демонстрации используем statsmodels и функцию het_breuschpagan
from statsmodels.compat import lzip
from statsmodels.stats.diagnostic import het_breuschpagan

# Генерируем остатки и "независимую переменную" (в данном случае просто случайные числа)
residuals = np.random.randn(100)
independent_var = np.random.randn(100).reshape(-1, 1)

bp_test = het_breuschpagan(residuals, independent_var)
lzip(["Lagrange multiplier statistic", "p-value", "f-value", "f p-value"], bp_test)


ValueError: The Breusch-Pagan test requires exog to have at least two columns where one is a constant.

In [9]:
def helwig_test(data):
    n = len(data)
    z = (data.mean() - 0) / (data.std(ddof=1) / np.sqrt(n))
    p_value = 2 * (1 - stats.norm.cdf(abs(z)))
    return z, p_value

# Тест на автокорреляцию (Durbin-Watson test)
def durbin_watson_test(residuals):
    num = np.sum(np.diff(residuals) ** 2)
    den = np.sum(residuals ** 2)
    return num / den

# Тест на гетероскедастичность (Breusch-Pagan test)
def breusch_pagan_test(y, x):
    import statsmodels.api as sm
    from statsmodels.stats.diagnostic import het_breuschpagan
    residuals = y - x.dot(sm.OLS(y, x).fit().params)
    bp_test = het_breuschpagan(residuals, x)
    return bp_test
# Тест Хельвига
helwig_stat, helwig_p_value = helwig_test(sample)

# Тест на автокорреляцию (применяем к случайной выборке как пример)
dw_stat = durbin_watson_test(sample)

# Тест на гетероскедастичность (требует зависимую и независимую переменные; пример на основе случайных данных)
y = np.random.randn(100, 1)
X = np.hstack([np.ones_like(y), np.random.randn(100, 1)])
bp_stat, bp_p_value, _, _ = breusch_pagan_test(y, X)

helwig_stat, helwig_p_value, dw_stat, bp_stat, bp_p_value


ValueError: resid is required to have ndim 1 but has ndim 2